In [4]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# Set up your webdriver
options = Options()
# Uncomment for headless mode
# options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

# Function to log in to LinkedIn
def linkedin_login(email, password):
    driver.get("https://www.linkedin.com/login")
    time.sleep(2)
    email_input = driver.find_element(By.ID, 'username')
    email_input.send_keys(email)
    password_input = driver.find_element(By.ID, 'password')
    password_input.send_keys(password)
    sign_in_button = driver.find_element(By.XPATH, '//button[@type="submit"]')
    sign_in_button.click()
    time.sleep(5)

# Function to get profile data
# Read CSV and scrape data
def scrape_from_csv(csv_file_path, email, password):
    linkedin_login(email, password)
    
    with open(csv_file_path, mode='r', newline='') as file:
        reader = csv.DictReader(file)
        for row in reader:
            profile_url = row['ProfileURL']  # Ensure your CSV has a column named 'ProfileURL'
            data = get_profile_data(profile_url)
            print(data)
# Function to get profile data
def get_profile_data(profile_url):
    driver.get(profile_url)
    time.sleep(5)  # Allow time for the page to load
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Extract the name and headline
    name = soup.find('h1').get_text().strip() if soup.find('h1') else "N/A"
    headline = soup.find('div', class_='text-body-medium break-words').get_text().strip() if soup.find('div', class_='text-body-medium break-words') else "N/A"
    
    # Experience Section Parsing
    experiences = []
    experience_divs = soup.find_all('div', class_='display-flex flex-column full-width')
    for item in experience_divs:
        company_info = item.find('span', class_='t-14 t-normal')
        company = company_info.get_text().strip() if company_info else "N/A"
        duration_info = item.find('span', class_='pvs-entity__caption-wrapper')
        duration = duration_info.get_text().strip() if duration_info else "N/A"
        #location_info = item.find('span', class_='t-14 t-normal t-black--light')
        #location = location_info.get_text().strip() if location_info else "N/A"
        experiences.append((company, duration))
    
    # Check if experiences are empty
    if not experiences:
        experiences.append("Not placed")
    
    return {"Name": name, "Headline": headline, "Experiences": experiences}

# Example usage
email = 'abc@gmail.com'  # Replace with your LinkedIn email
password = 'abc@1245'  # Replace with your LinkedIn password
csv_file_path = "lpr.csv"

scrape_from_csv(csv_file_path, email, password)

# Close the driver
driver.quit()


{'Name': 'Subham Singh', 'Headline': 'Pursuing Data Science and Analytics course from Imarticus Learning.', 'Experiences': [('[24]7.ai · Full-time[24]7.ai · Full-time', 'Oct 2021 - Dec 2022 · 1 yr 3 mos'), ('Reliance Nippon Life Insurance · Full-timeReliance Nippon Life Insurance · Full-time', 'Sep 2018 - Jun 2021 · 2 yrs 10 mos')]}
{'Name': '', 'Headline': 'N/A', 'Experiences': ['Not placed']}
{'Name': 'Rushtha Sherin KT', 'Headline': '--', 'Experiences': ['Not placed']}
{'Name': 'Sampath Kumar P', 'Headline': 'Immediate Joiner | Data science | Data analytics', 'Experiences': [('N/A', 'N/A')]}
{'Name': 'Sooraj Krishna', 'Headline': 'Attended Calicut University, Thenhipalem, Malapuram', 'Experiences': ['Not placed']}
{'Name': 'TEJAS R', 'Headline': 'Recent Graduate in Data Science | Enthusiastic About Data-Driven Decision Making"', 'Experiences': ['Not placed']}
{'Name': 'Twayyibah CP', 'Headline': 'Studied at Loyola College', 'Experiences': [('Ola · Full-timeOla · Full-time', 'May 202